## Ejercicio 4

Se busca predecir si el tipo de fármaco que se debe administrar a un paciente afectado de rinitis alérgica es el habitual o no. Se dispone de información correspondiente a las historias clínicas de pacientes atendidos previamente. Las variables relevadas son las siguientes:

- **Age:** Edad
- **Sex:** Sexo
- **BP (Blood Pressure):** Presión sanguínea.
- **Cholesterol:** nivel de colesterol.
- **Na:** Nivel de sodio en la sangre.
- **K:** Nivel de potasio en la sangre.
- **Class:** Fármaco suministrado. Cada paciente ha sido medicado con un único fármaco de 5 posibles: DrugA, DrugB, DrugC, DrugX, DrugY

### a) 

Utilice el archivo **drug_train.csv** para entrenar un perceptrón que sea capaz de predecir si el tipo de fármaco que se debe administrar a un paciente afectado de rinitis alérgica es el habitual (suministro de DrugY) o no.

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Fuentes.ClassPerceptron import Perceptron

ALPHA = 0.05
N_ITER = 100

DATOS_DIR   = '../../Datos/'
train_data = pd.read_csv(DATOS_DIR + 'drugs_train.csv')

def label_encoding_preprocess_data(data: pd.DataFrame) -> pd.DataFrame:
    # Codificación one-hot para 'Sex'
    NewSeEstiraColumn = pd.get_dummies(
        data['Sex'], prefix='Sex', drop_first=True
    ).astype(int)
    data = pd.concat([NewSeEstiraColumn, data], axis=1)
    data.drop(['Sex'], axis=1, inplace=True)

    Map = {
        "BP": {
            "LOW": 0,
            "NORMAL": 1,
            "HIGH": 2,
        },
        "Cholesterol": {
            "NORMAL": 0,
            "HIGH": 1,
        }
    }
    data.replace(Map, inplace=True)

    return data

preprocessed_data = label_encoding_preprocess_data(train_data)

ppn = Perceptron(alpha=ALPHA, n_iter=N_ITER, random_state=1)

X_train = preprocessed_data[['Age', 'Sex_M', 'BP', 'Cholesterol']].values
Y_train = (preprocessed_data["Drug"] == "drugY").astype(int)
ppn.fit(X_train, Y_train)

/tmp/ipykernel_709131/3178348950.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(Map, inplace=True)


### b) 

Luego utilice el archivo **drug_test.csv** para medir la calidad del modelo.

In [41]:
test_data = pd.read_csv(DATOS_DIR + 'drugs_test.csv')

training_data_not_normalized = label_encoding_preprocess_data(test_data)
X_test = training_data_not_normalized[['Age', 'Sex_M', 'BP', 'Cholesterol']].values
T_test = (training_data_not_normalized["Drug"] == "drugY").astype(int)
Y_test = ppn.predict(X_test)

efectividad = 100.0*np.sum(Y_test==T_test)/len(Y_test)
iter = len(ppn.errors_)
print("\nDrugY clasificada con %6.2f%% de efectividad en %d iteraciones" % (efectividad, iter))



DrugY clasificada con  45.00% de efectividad en 100 iteraciones


/tmp/ipykernel_709131/3178348950.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(Map, inplace=True)


### c) 

Resuelva el problema:
- Numerizando los atributos ordinales utilizando dos representaciones diferentes: como entero único y de manera binaria (dummy).
- Normalizando de diferentes formas: Sin normalizar, normalización lineal, normalización con media y desvío.

##### Numerización como entero único (label-encoding)

In [42]:
# La numerización como entero único es la que hice en un principio. A partir de la misma normalizo de las distintas formas
print("\nLabel-encoding sin normalizar con %6.2f%% de efectividad en %d iteraciones" % (efectividad, iter))

# Normalización lineal (0 a 1)
X_train_lin = (X_train - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))
X_test_lin  = (X_test  - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))

ppn.fit(X_train_lin, Y_train)
Y_test_lin = ppn.predict(X_test_lin)
efectividad_lin = 100.0*np.sum(Y_test_lin==T_test)/len(T_test)
print("Label-encoding con normalización lineal: %6.2f%%" % efectividad_lin)

# --- CASO 3: Normalización estándar (z-score) ---
X_train_std = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
X_test_std  = (X_test  - X_train.mean(axis=0)) / X_train.std(axis=0)

ppn.fit(X_train_std, Y_train)
Y_test_std = ppn.predict(X_test_std)
efectividad_std = 100.0*np.sum(Y_test_std==T_test)/len(T_test)
print("Label-encoding con normalización estándar: %6.2f%%" % efectividad_std)


Label-encoding sin normalizar con  45.00% de efectividad en 100 iteraciones
Label-encoding con normalización lineal:  45.00%
Label-encoding con normalización estándar:  42.50%


##### Numerización de manera binaria (dummy/one-hot-encoding)

In [ ]:
def one_hot_preprocess_data(data: pd.DataFrame) -> pd.DataFrame:
    categorical_cols = ['Sex', 'BP', 'Cholesterol']
    dummies = pd.get_dummies(data[categorical_cols], drop_first=False).astype(int)
    data = data.drop(columns=categorical_cols)
    data = pd.concat([dummies, data], axis=1)
    return data

train_data_oh = one_hot_preprocess_data(train_data)
test_data_oh  = one_hot_preprocess_data(test_data)

# --- Definir X y T ---
X_train = train_data_oh.drop(columns=['Drug']).values
Y_train = (train_data_oh["Drug"] == "drugY").astype(int)

X_test = test_data_oh.drop(columns=['Drug']).values
T_test = (test_data_oh["Drug"] == "drugY").astype(int)

ppn = Perceptron(alpha=ALPHA, n_iter=N_ITER, random_state=1)

# --- CASO 1: Sin normalizar ---
ppn.fit(X_train, Y_train)
Y_test_raw = ppn.predict(X_test)
efectividad_raw = 100.0*np.sum(Y_test_raw==T_test)/len(T_test)
print("One-hot encoding sin normalizar: %6.2f%% de efectividad en %d iteraciones" % (efectividad_raw, len(ppn.errors_)))

# --- CASO 2: Normalización lineal (0 a 1) ---
X_train_lin = (X_train - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))
X_test_lin  = (X_test  - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))

ppn.fit(X_train_lin, Y_train)
Y_test_lin = ppn.predict(X_test_lin)
efectividad_lin = 100.0*np.sum(Y_test_lin==T_test)/len(T_test)
print("One-hot encoding con normalización lineal: %6.2f%%" % efectividad_lin)

# --- CASO 3: Normalización estándar (z-score) ---
X_train_std = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
X_test_std  = (X_test  - X_train.mean(axis=0)) / X_train.std(axis=0)

ppn.fit(X_train_std, Y_train)
Y_test_std = ppn.predict(X_test_std)
efectividad_std = 100.0*np.sum(Y_test_std==T_test)/len(T_test)
print("One-hot encoding con normalización estándar: %6.2f%%" % efectividad_std)


One-hot encoding sin normalizar:  45.00% de efectividad en 100 iteraciones
One-hot encoding con normalización lineal:  97.50%
One-hot encoding con normalización estándar:  95.00%


> **Conclusión:** El orden de los datos no se ve bien reflejado por el label-encoding al normalizar.